### This notebook demonstrates the capabilities of the DeterministicReranking algorithm.
The algorithm provides a way to construct balanced rankings of candidates based on precalculated scores.

*Based on: [Sahin Cem Geyik, Stuart Ambler, & Krishnaram Kenthapadi (2019). Fairness-Aware Ranking in Search & Recommendation Systems with Application to LinkedIn Talent Search. In Proceedings of the 25th ACM SIGKDD International Conference on Knowledge Discovery & Data Mining](https://doi.org/10.48550/arXiv.1905.01989).*

The notebook is organized as follows:
1. Introduction;
2. A toy example;
3. Application to the LawSchoolGPA dataset;
4. Theoretical background.

#### 1. Introduction

Ranking algorithms are at the core of search and recommendation systems used in, among others, hiring, college admissions, and web-searches. It is clear that algorithmic bias in such cases can create or amplify unacceptable discrimination based on race, gender, or other attributes. Thus, a way of constructing "fair" rankings is needed.

Algorithms presented here take as input a **dataset that has already been ranked** (scored), possibly by some other machine learning model, and order them in a way that satisfies specified **fairness requirements**, expressed in a form of a **distribution over the protected attributes**.

#### 2. A toy example

Let's say we have a collection of 10 balls: 5 red and 5 blue. We assign each of them a score from 0 to 100; however, for some reason the red balls have a higher average score than the blue ones.

In [7]:
import numpy as np
import pandas as pd

balls = pd.DataFrame([['r', 100],['r', 90],['r', 85],['r', 70],['b', 70],['b', 60],['b', 50],['b', 40],['b', 30],['r', 20]],
                     columns=['color', 'score'])

print(f"Red mean score: {np.mean(balls[balls['color'] == 'r']['score'])}")
print(f"Blue mean score: {np.mean(balls[balls['color'] == 'b']['score'])}")

Red mean score: 73.0
Blue mean score: 50.0


Now, say we want to take the 6 best balls based on their score. In real life, the need for this limited "sub-ranking" may arise for many reasons. For example, the landing page of our ball-selling website may only have space for 6 items. 

This is similar to the case presented in the original paper, where the algorithm is used to rank job-seekers for recruiters on LinkedIn.

In [8]:
balls.sort_values(by='score', ascending=False)[:6]

color  score
0     r    100
1     r     90
2     r     85
3     r     70
4     b     70
5     b     60

Of course, we notice that we have only 1 blue ball in this ranking, and it is in the last position! On one hand, it seems fair in terms of scores. However, we may want a more **equal representation** of different colors in the ranking. The possible motivations for that in real life are clear.

In our case, the color is the **protected attribute**, and the red balls are a **privileged class**.

We may get a fairer ranking using the `DeterministicReranking` class:

In [9]:
from aif360.datasets import RegressionDataset
from aif360.algorithms.postprocessing.deterministic_reranking import DeterministicReranking

In [10]:
# Initialize a RegressionDataset with color as the protected attribute and red as the privileged class.
balls_ds = RegressionDataset(df=balls, dep_var_name='score', protected_attribute_names=['color'], privileged_classes=[['r']])
# keep the un-normalized scores for clarity; RegressionDataset normalizes them to be from 0 to 1.
balls_ds.labels = np.transpose([balls['score']])

To initialize the DeterministicReranking class, we need to pass the **protected attribute values** of the priviliged and unprivileged groups.
We do that using a list of dictionaries for each group, with the **name of the attribute as the key**. 

In [11]:
# The RegressionDataset class automatically maps the priviliged attribute value (color='red') to 1 and the other to 0.
dr = DeterministicReranking(unprivileged_groups=[{'color': 0}], privileged_groups=[{'color': 1}])

Use the `fit_predict` method to get the ranking. The arguments are:
- `dataset` is the dataset to construct a ranking from;
- `rec_size` is the **size** of the ranking we need - in our case 6;
- `target_prop` is the **desired proportion** of items of each group in the ranking in the form of dictionary; the keys are the corresponding protected attribute values. We need equal representation, so we pass `{0: 0.5, 1: 0.5}`;
- `rerank_type` is the algorithm to use; for further details, skip to section 4 of this notebook. For now, we stick to the default `Constrained`;
- `renormalize_scores` will normalize the scores in the result so that the lowest is 0 and the highest is 1. Default is `False`.

In [12]:
fair_ranking = dr.fit_predict(dataset=balls_ds, rec_size=6, target_prop=[0.5, 0.5])
fair_ranking.convert_to_dataframe()[0]

color  score
0    1.0  100.0
1    1.0   90.0
4    0.0   70.0
2    1.0   85.0
5    0.0   60.0
6    0.0   50.0

In this result, the proportions are equal. Additionally, as the algorithm goes through positions in the ranking one-by-one, checking each time for violations of fairness, the items belonging to the unprivileged group (blue balls) **aren't all at the "bottom"** of the ranking.

#### 3. Application to the Law School GPA Dataset

We now apply the algorithm to a more serious example - the Law School GPA dataset.

The aim is to obtain a ranking of size 20 according to the value in `zfygpa` with fairness constraints on `race`.

In [13]:
import tempfile
import requests
import zipfile
import os
import pandas as pd 

with tempfile.TemporaryDirectory() as temp_dir:
    response = requests.get("http://www.seaphe.org/databases/LSAC/LSAC_SAS.zip")
    temp_file_name = os.path.join(temp_dir, "LSAC_SAS.zip")
    with open(temp_file_name, "wb") as temp_file:
        temp_file.write(response.content)
    with zipfile.ZipFile(temp_file_name, 'r') as zip_ref:
        zip_ref.extractall(temp_dir)
    data = pd.read_sas(os.path.join(temp_dir, "lsac.sas7bdat"))
    data = data.assign(gender=(data["gender"] == b"male") * 1)
    data['race'] = data['race1']
    data = data[['race', 'gender', 'lsat', 'ugpa', 'zfygpa']]


In [25]:
from aif360.datasets import RegressionDataset
dataset = RegressionDataset(data, dep_var_name='zfygpa', protected_attribute_names=['race'], privileged_classes=[[b'white']])

First, we take a ranking strictly according to the score.

In [26]:
df = dataset.convert_to_dataframe()[0].sort_values(by=['zfygpa'], ascending=False)
unfair_ranking = RegressionDataset(df=df, dep_var_name='zfygpa', protected_attribute_names=['race'], privileged_classes=[[1]])
# Again, for clarity we leave the labels (scores) unnormalized.
unfair_ranking.labels = np.transpose([df['zfygpa']])

Problem: **all the items come from the priviliged group!**

In [27]:
rank = unfair_ranking.convert_to_dataframe()[0][:20]
rank

race  gender      lsat   ugpa    zfygpa
5537    1.0     1.0  0.716216  1.000  1.000000
25381   1.0     1.0  0.675676  0.750  0.970960
9699    1.0     1.0  0.891892  1.000  0.960859
6160    1.0     1.0  0.783784  0.750  0.953283
13478   1.0     1.0  0.783784  0.700  0.953283
21127   1.0     0.0  0.837838  0.875  0.952020
19475   1.0     0.0  0.891892  0.950  0.948232
24329   1.0     1.0  0.729730  0.525  0.944444
11152   1.0     1.0  1.000000  0.975  0.944444
16978   1.0     0.0  0.675676  1.000  0.930556
7525    1.0     1.0  0.864865  0.850  0.924242
48      1.0     0.0  0.756757  1.000  0.924242
1600    1.0     0.0  0.891892  1.000  0.920455
20773   1.0     1.0  0.972973  0.750  0.920455
22047   1.0     1.0  1.000000  0.850  0.920455
16428   1.0     1.0  0.756757  0.875  0.920455
17585   1.0     1.0  0.702703  0.600  0.916667
8119    1.0     0.0  0.594595  0.700  0.912879
11738   1.0     1.0  0.729730  0.875  0.911616
10563   1.0     1.0  1.000000  0.900  0.909091

Try to get a fair ranking using `DeterministicReranking`:

In [28]:
dr = DeterministicReranking(unprivileged_groups=[{'race': 0}], privileged_groups=[{'race': 1}])
fair_ranking = dr.fit_predict(dataset, rec_size=20, target_prop=[0.5, 0.5], rerank_type='Constrained')
fair_ranking.convert_to_dataframe()[0]

race  gender      lsat      ugpa    zfygpa
5537    1.0     1.0  0.637931  1.000000  1.000000
25381   1.0     1.0  0.586207  0.473684  0.970960
20893   0.0     0.0  0.000000  0.842105  0.900253
9699    1.0     1.0  0.862069  1.000000  0.960859
25021   0.0     0.0  0.620690  0.105263  0.888889
6160    1.0     1.0  0.724138  0.473684  0.953283
3055    0.0     1.0  0.689655  0.421053  0.888889
13478   1.0     1.0  0.724138  0.368421  0.953283
24500   0.0     1.0  0.965517  0.947368  0.887626
21127   1.0     0.0  0.793103  0.736842  0.952020
26383   0.0     0.0  1.000000  0.736842  0.864899
19475   1.0     0.0  0.862069  0.894737  0.948232
12816   0.0     1.0  0.810345  0.842105  0.856061
24329   1.0     1.0  0.655172  0.000000  0.944444
25925   0.0     0.0  0.827586  0.736842  0.853535
11152   1.0     1.0  1.000000  0.947368  0.944444
6287    0.0     1.0  0.655172  0.736842  0.849747
16978   1.0     0.0  0.586207  1.000000  0.930556
17964   0.0     0.0  0.724138  0.473684  0.849747
5951    0.0     0.0  0.482759  0.210526  0.848485

This seems much better.

To get a better idea of the level of fairness, we can quantify it using the **Infeasible Index** metric, which measures the number of indices at which the fairness requirements are violated. For a ranking of size $r$ and groups of items $g_i \in G$ with desired proportions $p_i$ it is defined as follows:

$$InfeasibleIndex_r = \sum_{k=0}^{r} 1(\exists\ group\ g_i \in G: count_k(g_i) < \lfloor(p_i*k)\rfloor)$$

Let us calculate the InfeasibleIndex using the `infeasible_index` method of the `RegressionDatasetMetric` class, first for the strictly score-based ranking, and then for the fair ranking.

It returns the value of the metric, as well as positions in the ranking at which the fairness constraints are violated.

In [29]:
from aif360.metrics.regression_metric import RegressionDatasetMetric

In [30]:
m = RegressionDatasetMetric(dataset=unfair_ranking, unprivileged_groups=[{'race': 0}], privileged_groups=[{'race': 1}])
m.infeasible_index(target_prop={0: 0.5, 1: 0.5}, r=20)

(18, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18])

In [31]:
# The II for this ranking is much better!
m_fair = RegressionDatasetMetric(dataset=fair_ranking, unprivileged_groups=[{'race': 0}], privileged_groups=[{'race': 1}])
m_fair.infeasible_index(target_prop={0: 0.5, 1: 0.5}, r=20)

(9, [1, 3, 5, 7, 9, 11, 13, 15, 17])

We can now quantify the loss in the quality of the ranking (how "out of order" the items are, based on their scores) induced by the reranking algorithm. For this we can use the **Discounted Cumulative Gain**, defined as follows:

$$DCG@k = \sum_{j=0}^{k} \frac{score(j)}{log_2(j+1)}$$

where $score(j)$ denotes the score of the item at position $j$.

This metric can also be normalized against the value of the "perfect" strictly score-based ordering, giving it a range from 0 to 1.

In [32]:
print(f'Normalized DCG of strictly score-based ranking: {m.discounted_cum_gain(normalized=True)}')
print(f'Normalized DCG of fair ranking: {m_fair.discounted_cum_gain(normalized=True)}')

Normalized DCG of strictly score-based ranking: 1.0
Normalized DCG of fair ranking: 0.9473492630864591


#### 4. Variations of the algorithm

The `predict` and `fit_predict` methods of the algorithm include a `rerank_type` parameter. It refers to the different algorithms described in the original paper: **Greedy**, **Conservative**, **Relaxed** and **Constrained**.